In [1]:
# 1) Create the inverse modeling mesh for time lapse inversion

import subprocess
import os
import sys

os.chdir('../mesh')

cmd='mpirun -np 1 e4d' # assumes mpirun and e4d are in your path
result = subprocess.run(cmd, shell=True,stdout=subprocess.PIPE)
output=result.stdout.decode("utf-8") 

lines=output.split(sep='\n')
cont=False
for line in lines:
        #print (line)
        if 'Finished build' in line:
            print ('Mesh build okay')
            cont=True

# copy the mesh files to the main directory and remove mesh generation files
cmd='mv sp.1* ../time_lapse/.'
subprocess.run(cmd, shell=True,stdout=subprocess.PIPE)
cmd='mv sp.trn ../time_lapse/.'
subprocess.run(cmd, shell=True,stdout=subprocess.PIPE)

#cmd="rm -v !('e4d.inp'|'sp.cfg') "
#subprocess.run(cmd, shell=True,stdout=subprocess.PIPE)
cmd='rm surface.*'
subprocess.run(cmd, shell=True,stdout=subprocess.PIPE)
cmd='*.poly*'
subprocess.run(cmd, shell=True,stdout=subprocess.PIPE)
cmd='rm mesh_build.log'
subprocess.run(cmd, shell=True,stdout=subprocess.PIPE)
cmd='rm e4d.log'
subprocess.run(cmd, shell=True,stdout=subprocess.PIPE)
cmd='rm sp.sig'
subprocess.run(cmd, shell=True,stdout=subprocess.PIPE)
cmd='rm sp.h5'
subprocess.run(cmd, shell=True,stdout=subprocess.PIPE)
cmd='rm sp.xmf'
subprocess.run(cmd, shell=True,stdout=subprocess.PIPE)
cmd='rm *poly*'
subprocess.run(cmd, shell=True,stdout=subprocess.PIPE)


os.chdir('../time_lapse')

Mesh build okay


In [17]:
#2) Move over the baseline solution to use as a starting and reference model in the time-lapse inversion

# THE BASELINE INVERSION MUST BE RUN FIRST TO USE THIS CODE
# Query /baseline and use the last sigma.* file as the baseline solution.
# This is the sigma file with the largest integer
import os
import re
import subprocess
files=os.listdir('../baseline')

iter=0
for sigF in files:
    if (sigF.startswith("sigma.")):
        tmp=re.findall('\d+', sigF )
        tmp = [int(x) for x in tmp]
        
        if tmp[0] > iter:
            iter=tmp[0]
            
fn='sigma.'+str(iter)

cmd='cp ../baseline/'+fn+ ' baseline_sp.sig'
subprocess.run(cmd, shell=True,stdout=subprocess.PIPE)


CompletedProcess(args='cp ../baseline/sigma.3 baseline_sp.sig', returncode=0, stdout=b'')

In [18]:
#3) Create surveys.txt file

# THE FORWARD MODELS MUST BE CREATED BEFORE RUNNING THIS CODE
# This can be done by extracting surveys.tgz or
# running forward/ERT2_sp.ipynb

import os
import re
import subprocess

files=os.listdir('.')


surv_fn=[]
num=[]
iter=0
for sigF in files:
    if (sigF.startswith("sig") and sigF.endswith(".sig.srv")):
        tmp=re.findall('\d+', sigF )
        tmp = [int(x) for x in tmp]
        num.append(float(str(tmp[0])+'.'+str(tmp[1])))
num = sorted(num)
print (num)  

outfile = open('surveys.txt','w+')
outfile.write('{}\n'.format(str(len(num))))
for i in range(len(num)):
    outfile.write('{:20} {:5}\n'.format('sig_'+str(num[i])+'.sig.srv',num[i]))

print ('done')

[0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0]
done


In [23]:
# Create e4d.inp file, using the baseline solution reference model
### THESE FILES ARE PROVIDED IN /inv_previous

pre='sp'
srvfile='two_blocks.srv'
outoptFile='two_blocks.out'

fileN='e4d.inp'
f1=open(fileN, 'w')
f1.write("ERT4\n")              # run mode ERT4
f1.write(pre+".1.node\n")       # mesh node file
f1.write("baseline.sig.srv\n")  # baseline survey file
f1.write('baseline_sp.sig\n')   # starting conductivity file
f1.write(pre+'.out\n')          # output options file
f1.write(pre+'_tl.inv\n')       # inversion options file
f1.write('baseline_sp.sig\n')   # reference model file
f1.write('surveys.txt 1\n')     # survey file list, using the baseline solution as the starting model
f1.close()


# check e4d.log for progress 
import subprocess

cmd='/home/robi526/codes/e4d_dev/petsc/arch-linux-c-opt/bin/mpirun -np 20 e4d' 
result = subprocess.run(cmd, shell=True,stdout=subprocess.PIPE)
output=result.stdout.decode("utf-8") 


#4) Run the inversion, using a baseline reference model
### THESE FILES ARE PROVIDED IN /inv_baseline

# check e4d.log for progress 
import subprocess

cmd='/home/robi526/codes/e4d_dev/petsc/arch-linux-c-opt/bin/mpirun -np 20 e4d' # assumes mpirun and e4d are in your path
result = subprocess.run(cmd, shell=True,stdout=subprocess.PIPE)
output=result.stdout.decode("utf-8") 



In [15]:
# 5) OPTIONAL:  Build the px file to view the time-lapse results
import subprocess

# get the output file names from surveys.txt
f = open("surveys.txt","r") 

first=True
for line in f:
    try: 
        int(line)
    except:
        tmp=line.split()

        tl_fn='tl_sig'+str(tmp[1])+'00'
        print (tl_fn)

        if first==True:
            cmd='px -f sp '+tl_fn+' tl_sp '+str(tmp[1])
            first=False
        else:
            cmd='px -af sp '+tl_fn+' tl_sp '+str(tmp[1])

        result = subprocess.run(cmd, shell=True,stdout=subprocess.PIPE)
        output=result.stdout.decode("utf-8") 

f.close()

tl_sig0.000
tl_sig0.500
tl_sig1.000
tl_sig1.500
tl_sig2.000
tl_sig2.500
tl_sig3.000
tl_sig3.500
tl_sig4.000
tl_sig4.500
tl_sig5.000
tl_sig5.500
tl_sig6.000
tl_sig6.500
tl_sig7.000
tl_sig7.500
tl_sig8.000
tl_sig8.500
tl_sig9.000
tl_sig9.500
tl_sig10.000


In [21]:
#6) Modify the e4d.inp file, using a previous solution reference model
### THESE FILES ARE PROVIDED IN /inv_previous

pre='sp'
srvfile='two_blocks.srv'
outoptFile='two_blocks.out'

fileN='e4d.inp'
f1=open(fileN, 'w')
f1.write("ERT4\n")              # run mode ERT4
f1.write(pre+".1.node\n")       # mesh node file
f1.write("baseline.sig.srv\n")  # baseline survey file
f1.write('baseline_sp.sig\n')   # starting conductivity file
f1.write(pre+'.out\n')          # output options file
f1.write(pre+'_tl.inv\n')       # inversion options file
f1.write('baseline_sp.sig\n')   # reference model file
f1.write('surveys.txt 2\n')     # survey file list, using the previous solution as the starting model
f1.close()


# check e4d.log for progress 
import subprocess

cmd='/home/robi526/codes/e4d_dev/petsc/arch-linux-c-opt/bin/mpirun -np 20 e4d' 
result = subprocess.run(cmd, shell=True,stdout=subprocess.PIPE)
output=result.stdout.decode("utf-8") 




KeyboardInterrupt: 

In [ ]:
# 5) OPTIONAL:  Build the px file to view the time-lapse results
import subprocess

# get the output file names from surveys.txt
f = open("surveys.txt","r") 

first=True
for line in f:
    try: 
        int(line)
    except:
        tmp=line.split()

        tl_fn='tl_sig'+str(tmp[1])+'00'
        print (tl_fn)

        if first==True:
            cmd='px -f sp '+tl_fn+' tlprev_sp '+str(tmp[1])
            first=False
        else:
            cmd='px -af sp '+tl_fn+' tlprev_sp '+str(tmp[1])

        result = subprocess.run(cmd, shell=True,stdout=subprocess.PIPE)
        output=result.stdout.decode("utf-8") 

f.close()